## Imports

In [1]:
import torch

import networkx as nx

from sklearn.model_selection import train_test_split

In [2]:
from utils.corpus_utils import Corpus, CorpusMaker

## Data opening

In [3]:
filepath = 'data/Bunin-Temnye_allei.txt'

In [4]:
with open(filepath, 'r', encoding='cp1251') as f:
    corpus_raw = f.read().lower()

In [5]:
corpus_raw = corpus_raw.split('\n')[57:57+20]

In [6]:
train_corpus_raw, test_corpus_raw = train_test_split(corpus_raw, test_size=0.1)

In [7]:
test_corpus_raw[0]

'—\xa0баба — ума палата. и все, говорят, богатеет. деньги в рост даёт.'

In [8]:
import pandas as pd

df = pd.read_csv('../russe-wsi-kit/data/main/wiki-wiki/train.csv', delimiter='\t')
train_corpus_raw = train_corpus_raw + list(df['context'])

## Texts to graph

In [9]:
corpuser = CorpusMaker()

In [10]:
train_graph = corpuser.create_graph(train_corpus_raw)

In [11]:
test_graph = corpuser.create_graph(test_corpus_raw)#, dicts=(train_graph.idx2token, train_graph.token2idx))

## Saving

In [12]:
torch.save(train_graph, 'data/train.pickle')
torch.save(test_graph, 'data/test.pickle')

In [14]:
## Vizualization

In [15]:
# G = nx.Graph(train_graph['edge_index'].tolist())

In [16]:
# nx.write_gexf(G, "test.gexf")

In [17]:
# nx.draw_networkx(G)